# Finetuning Using Google Gemma's Model

In [ ]:
#accelerate

In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 55.5 MB/s eta 0:00:00


In [8]:
pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [2]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [3]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

Defining model wiht QLORA

nf4(4-bit NormalFloat(NF4))

In [4]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Loading Autotokenizer and Casual Language model

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Checking without fine tuning the model

In [14]:
text = "Book title: Le Démon de la Colline aux Loups. Author:  Summary: "
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Book title: Le Démon de la Colline aux Loups. Author:  Summary: 

The story of a young man who is haunted by the ghost of a woman who died in a


In [16]:
text = "Book title: Le Démon de la Colline aux Loups. Author and Summary: "
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Book title: Le Démon de la Colline aux Loups. Author and Summary: 19th century French author, Victor Hugo. Summary: The story is about a man who is a very rich man. He is very rich because he has a lot of money. He is very rich because he has a lot of land. He


In [9]:
os.environ["WANDB_DISABLED"] = "false"

FineTuning the Model

In [10]:
lora_config = LoraConfig(
    r = 2,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [17]:
from datasets import load_dataset

data = load_dataset("Abirate/french_book_reviews")
data = data.map(lambda samples: tokenizer(samples["book_title"]), batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/9658 [00:00<?, ? examples/s]

In [21]:
data['train']['book_title'][2]

'La plus secrète mémoire des hommes'

In [22]:
def formatting_func(example):
    text = f"book_title: {example['book_title'][0]}\author: {example['author'][0]}\reader_review: {example['reader_review'][0]}"
    return [text]

In [24]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=50,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

In [25]:
trainer.train()

Step,Training Loss
1,5.340200
2,4.379600
3,5.182600
4,4.272600
5,5.398800
6,5.021700
7,3.736700
8,4.178000
9,4.072800
10,3.846900


TrainOutput(global_step=50, training_loss=2.5487073624134062, metrics={'train_runtime': 83.3931, 'train_samples_per_second': 2.398, 'train_steps_per_second': 0.6, 'total_flos': 166207972638720.0, 'train_loss': 2.5487073624134062, 'epoch': 20.0})

In [28]:
text = "book_title: La grande course de Flanagan. author: Tom McNab"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

eader_review: J'ai adoré ce livre ! J'ai adoré le style de narration, j'ai adoré le personnage principal, j'ai adoré le début de l'histoire ! J'ai adoré tout


In [29]:
text = "book_title: L'incroyable histoire du coq qui ne voulait pas fermer son bec. author: Thomas Gerbeaux"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

eader_review: J'ai adoré ce livre ! J'ai adoré ce coq ! J'ai adoré ce tome ! J'ai adoré ce tome 2 ! J'ai adoré ce tome 3 ! J
